installing some stuff

In [2]:
!pip install bm25s
!pip install anvil-uplink

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.4 MB/s eta 0:00:00


In [3]:
import bm25s
import requests
import anvil.server
import pandas as pd

Im using anvil as my gui, as it is pretty simple to implement, and the focus is not on the gui, but on the search and crawling functionality

In [4]:
anvil.server.connect("server_YRVCVOKMEVJDVSNRL5YQQPUI-QQGZVPGSKMYPC74L")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER


In [92]:
df = pd.read_csv("https://raw.githubusercontent.com/bshort95/AON_Scraper/refs/heads/main/corpusData.csv")
df = df.fillna("")
df["searchDoc"] = df["name"] + "|"+ df["description"] +"|"+ df["addInfo"]
corpus = df["searchDoc"].values.tolist()

using the bm25



In [96]:
retriever = bm25s.BM25(corpus=corpus)
retriever.index(bm25s.tokenize(corpus))

Split strings:   0%|          | 0/1419 [00:00<?, ?it/s]

DEBUG:bm25s:Building index from IDs objects


BM25S Count Tokens:   0%|          | 0/1419 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/1419 [00:00<?, ?it/s]

this function returns a more user friendly response to the bm25 query

In [110]:
def getCreatureBlock(name):
  temp = df.loc[df['name'] == name].to_dict(orient='records')
  output = temp[0]["level"] +" | " + temp[0]["name"] + "\n---------\n" + temp[0]["description"]+"\nhttps://2e.aonprd.com/Monsters.aspx?ID="+str(temp[0]["id"])+ "\n__________\n"
  return output

In [104]:
@anvil.server.callable
def searchCorpus(query, k):
  textTotals = ""
  results, scores = retriever.retrieve(bm25s.tokenize(query), k=k)
  resultList = results[0]
  for x in range(k):
    textTotals = textTotals + getCreatureBlock(resultList[x].split("|")[0]) + "\n"
  return textTotals

In [109]:
anvil.server.wait_forever()

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: 